In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import re
import time

In [19]:
options = EdgeOptions()
options.use_chromium = True
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.set_capability("acceptInsecureCerts", True)
#Try to connect if the edgeriver version is 108
options.binary_location =r'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'

In [20]:
dicParfum={}
listeMarque=[]
listeIngredient=[]

In [21]:
driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
driver.get("https://www.sephora.fr/")
cookies=driver.find_element(By.XPATH,'//*[@id="footer_tc_privacy_button_2"]')
cookies.click()
for i in range (0,10):
    div=0
    driver.get("https://www.sephora.fr/parfum-meilleures-ventes/?srule=Sorting%20Rule%20-%20Best%20Sellers&start=0&sz=24")
    id=str(i+1)
    #print("top : "+ id )
    try:
        pathparfum="/html/body/div[2]/div[3]/div[2]/div[3]/div[2]/div[2]/ul/li["+id+"]"
        #//*[@id="search-result-items"]/li[1]/div
        Parfum=driver.find_element(By.XPATH, pathparfum)
    except :
        pathparfum="/html/body/div[1]/div[3]/div[2]/div[3]/div[2]/div[2]/ul/li["+id+"]"
        Parfum=driver.find_element(By.XPATH, pathparfum)
    if Parfum.get_attribute('class')=='grid-tile':
        marque=driver.find_element(By.XPATH,pathparfum+"/div/div[3]/div/a/span").get_attribute('innerText')
        #print(marque)
        listeMarque.append(marque)
        Parfum.click()
        #nom du parfum      
        nomParfum=driver.find_element(By.CLASS_NAME,'product-name.product-name-bold').get_attribute("innerText")
        #print(nomParfum)
        dicParfum[i]=nomParfum
        #liste des ingredients
        ingredient=driver.find_element(By.CLASS_NAME,'ingredients-content.tabs-panel').get_attribute('innerText')
        #print('Ingredient : '+ ingredient)
        listeIngredient.append(ingredient)
        #print("\n")
driver.close()
        

<ipython-input-21-47e148e7b33a>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)


# Gestion des Ingredients

In [22]:
phrase="Cette liste d'ingrédients peut faire l'objet de modifications, veuillez consulter l'emballage du produit acheté."
Ingredient=[]
for i in listeIngredient:
    ouisticram=i.replace("\n",'')
    ouisticram=ouisticram.replace(phrase,'')
    ousticram=ouisticram.replace(phrase,'')
    if "INGREDIENTS" in ouisticram:
        champenfeu=ouisticram.split(':')
        champenfeu=champenfeu[-1]
    else:
        champenfeu=ouisticram
    if "●" in champenfeu :
        simiabraz=champenfeu.split("●")
    elif " | " in champenfeu:
        simiabraz=champenfeu.split(" | ")
    elif ' ,' in champenfeu:
        simiabraz=champenfeu.split(" ,")
    elif ',' in champenfeu:
        simiabraz=champenfeu.split(",")
    elif " • " in champenfeu:
        simiabraz=champenfeu.split(" • ")
    else:
        print(simiabraz)
    Ingredient.append(simiabraz)
    

In [23]:
Ingredients=[]
for i in Ingredient:
    for j in i :
        if j.startswith('#'):
            ij=j.split(' ')
            j=ij[1]
        if j.startswith(' '):
            j=j[1:]
        if j.endswith(' '):
            j=j[:-1]
        Ingredients.append(j)

In [24]:
setIngredient=set(Ingredients)

In [25]:
descIng={}
Allergene=[]
endo=[]

In [26]:
%%time
driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
driver.get("https://incibeauty.com/ingredients")
cookies=driver.find_element(By.XPATH,"/html/body/div[6]/div[1]/div/div[2]/button")
cookies.click()
for i in setIngredient:
    print(i)
    driver.get("https://incibeauty.com/ingredients")
    search=driver.find_element(By.ID,'searchInci')
    search.click()
    search.clear()
    search.send_keys(i)
    count=0
    url=False
    b=True
    while b:
        count+=1
        search.send_keys(Keys.ENTER)
        if driver.current_url!="https://incibeauty.com/ingredients":
            b=False
            url=True
        if count>=10:
            b=False
    if url :
        listeInfo= WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/section[1]/div/div/div/ul')))
        if 'Allergène' in listeInfo.get_attribute('innerText'):
            Allergene.append(i)
        if 'Perturbateur endocrinien' in listeInfo.get_attribute('innerText'):
            endo.append(i)
        try:
            aSavoir=driver.find_element(By.XPATH,'/html/body/section[1]/div/div/div/div[2]/div')
            descIng[i]=aSavoir.get_attribute('innerText')
        except:
            descIng[i]="Pas d'information supplémentaire"

driver.close()

<timed exec>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3


POLYACRYLATE CROSSPOLYMER-6
TOCOPHEROL
GERANIOL
PARFUM (FRAGRANCE)
BENZYL BENZOATE
METHYL ANTHRANILATE
CI 60730 (EXT. VIOLET 2)
TRIETHYL CITRATE
CI 15985 (YELLOW 6)
ETHYLHEXYL METHOXYCINNAMATE
BENZYL SALICYLATE
FARNESOL
AQUA / WATER / EAU
HEXYL CINNAMAL
DIETHYLAMINO HYDROXYBENZOYL HEXYL BENZOATE
ALCOHOL DENAT.
IL60-1
AQUA (WATER)
COUMARIN
PENTAERYTHRITYL TETRA-DI-T-BUTYL HYDROXYHYDROCINNAMATE
LINALOOL
IL95-1
EVERNIA PRUNASTRI (OAK MOSS) EXTRACT
PARFUM / FRAGRANCE
CITRUS AURANTIUM AMARA (BITTER ORANGE) FLOWER WATER
HYDROXYCITRONELLAL
BENZYL ALCOHOL
BUTYL METHOXYDIBENZOYLMETHANE
FARNESOLCITRAL
ALCOHOL
ISOEUGENOL
EUGENOL
CI 42090 (BLUE 1)
BENZYL CINNAMATE
CI 17200 (RED 33)
CI 17200 / RED 33 (F.I.L. N292437/1).
IL47-1
TRIS(TETRAMETHYLHYDROXYPIPERIDINOL) CITRATE
CITRONELLOL
AMMONIUM ACRYLOYLDIMETHYLTAURATE/BEHENETH-25 METHACRYLATE CROSSPOLYMER
ETHYLHEXYL SALICYLATE
BHT
CI 14700 (RED 4)
CINNAMYL ALCOHOL
LIMONENE
CI 19140 (YELLOW 5)
T-BUTYL ALCOHOL
CITRAL
ALPHA-ISOMETHYL IONONE
ANISE ALCOHOL


In [27]:
driver.quit()

In [28]:
Allergene

['GERANIOL',
 'BENZYL BENZOATE',
 'FARNESOL',
 'ALCOHOL DENAT.',
 'COUMARIN',
 'LINALOOL',
 'EVERNIA PRUNASTRI (OAK MOSS) EXTRACT',
 'CITRUS AURANTIUM AMARA (BITTER ORANGE) FLOWER WATER',
 'HYDROXYCITRONELLAL',
 'BENZYL ALCOHOL',
 'ALCOHOL',
 'ISOEUGENOL',
 'EUGENOL',
 'BENZYL CINNAMATE',
 'LIMONENE',
 'ALPHA-ISOMETHYL IONONE']

In [29]:
descIng

{'POLYACRYLATE CROSSPOLYMER-6': "À SAVOIR\nLe polysorbate 20 ou encore Tween 20 est tensioactif non ionique et un émulsifiant dérivé de sorbitol à base de fruits et de baies. Il permet de diluer les huiles essentielles ou parfum dans les produits cosmétiques à base d'eau comme les lotions, crèmes, gels.\nLe polysorbate 20 est produit par éthoxylation. Le nombre 20 indique le nombre moyen de moles d'oxyde d'éthylène ayant réagi par mole de sorbitol.",
 'GERANIOL': "À SAVOIR\nLe Géraniol aussi nommé rhodinol, est un alcool monoterpénique qui fait partie des 24 allergènes réglementés par l'Europe. On le retrouve présent dans les huiles essentielles de Géranium, de citronnelle mais aussi dans l'huile de rose et de Palmarosa. Il est utilisé en parfumerie pour son odeur de rose.",
 'PARFUM (FRAGRANCE)': "À SAVOIR\nDerrière ce terme de parfum, se cachent malheureusement de nombreuses molécules odorantes potentiellement irritantes voire allergisantes. Difficile de s'y retrouver lorsque que l'o

# Gestion des marques

In [30]:
marqueEco={listeMarque[i]+' ': 'Sans engagement' for i in range(len(listeMarque))}
marqueEco

{'GIVENCHY ': 'Sans engagement',
 'DIOR ': 'Sans engagement',
 'CHANEL ': 'Sans engagement',
 'PACO RABANNE ': 'Sans engagement',
 'PRADA ': 'Sans engagement'}

In [35]:
driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
driver.get("https://sciencebasedtargets.org/companies-taking-action#dashboard")
time.sleep(5)
cookies=driver.find_element(By.CLASS_NAME,"font-bold.text-center.rounded-full.appearance-none.o-btn.border-3.border-currentColor.flex-shrink-0.is-current")
cookies.click()
perpage=Select(driver.find_element(By.XPATH,'//*[@id="per-page"]'))
perpage.select_by_value("50")
while True:
    #try:
        for ids in range (1,11):
            table=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[1]/div")
            driver.execute_script("window.scrollTo(0, 10)")
            b=True
            while b:
                count+=1
                driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[2]/label").click()
                if driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[2]/div/div[1]/div[4]").isDisplayed():
                    b=False
                if count>=10:
                    b=False
            sectors=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[2]/div/div[1]/div[4]")
            print(sectors.get_attribute('innerText'))
            for case in table:
                if case.get_attribute('innerText').split('\n')[0] in marqueEco.keys():
                    print('x')
        myElem=driver.find_element(By.XPATH,'/html/body/main/div/div[3]/div[2]/div/div/div/div[3]/div/button[2]')
        myElem.click()
    #except:
        #break

<ipython-input-35-b583628f7c0b>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)


AttributeError: 'WebElement' object has no attribute 'isDisplayed'

In [ ]:
marqueEco